# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: AI-generated content such as articles, social media posts, and product descriptions can help businesses save time and resources while maintaining consistency in tone and style.
2. **Product Design**: Generative AI can be used to design new products, such as furniture, electronics, or clothing, using complex algorithms that learn from patterns and trends in data.
3. **Marketing Automation**: AI-generated content, such as email campaigns and social media ads, can help businesses personalize their marketing efforts and reach a wider audience.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answering frequently asked questions and routing complex issues to human representatives.
5. **Financial Analysis**: AI-generated financial models and reports can help businesses analyze market trends, forecast revenue, and make informed inv

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, blog posts, social media posts, product descriptions, and even entire books. This can help businesses save time and resources while maintaining consistency in their content.
2. **Product Design**: Generative AI can design products such as furniture, clothing, and jewelry by creating 3D models and simulations of different designs. This can help businesses reduce the time and cost associated with traditional product design processes.
3. **Marketing and Advertising**: Generative AI can generate personalized ads, social media posts, and email campaigns that are tailored to individual customers' interests and preferences.
4. **Customer Service**: Generative AI-powered chatbots can provide 24/7 customer support, helping businesses to improve customer satisfaction and reduce the workload of human customer service rep

## Alternative approach - using OpenAI python library to connect to Ollama

In [7]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI, also known as Generation AI or Generative Model, is a type of artificial intelligence that can generate new data, such as images, text, or audio, based on existing data. Here are some potential business applications of Generative AI:

1. **Content Creation**: Generative AI can be used to automate content creation, such as generating blog posts, social media content, or product descriptions.
2. **Image and Video Generation**: Generative AI can generate high-quality images and videos for various purposes, such as product visualization, advertising, or entertainment.
3. **Chatbots and Virtual Assistants**: Generative AI can be used to create chatbots that can understand and respond to customer inquiries in a more natural and human-like way.
4. **Recommendation Systems**: Generative AI can be used to develop recommendation systems that suggest products or services based on customer behavior and preferences.
5. **Data Augmentation**: Generative AI can be used to generate new 

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [8]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']

In [14]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [15]:
display_summary("https://edwarddonner.com")

### Summary of Edward Donner's Website

#### Overview
The website is a personal blog and professional platform for Edward Donner, a technology entrepreneur and AI enthusiast.

#### Projects and Ventures
* Co-founder and CTO of Nebula.io, an AI startup that applies AI to help people discover their potential.
* Previous founder and CEO of untapt, an AI startup acquired in 2021.

#### News and Announcements

* **Mastering AI and LLM Engineering – Resources** (November 13, 2024): A collection of resources for mastering AI and LLM engineering.
* **From Software Engineer to AI Data Scientist – resources** (October 16, 2024): A collection of resources for transitioning from software engineer to AI data scientist.
* **Outsmart LLM Arena – a battle of diplomacy and deviousness** (June 26, 2024): An arena that pits LLMs against each other in a battle of diplomacy and deviousness.
* **Choosing the Right LLM: Toolkit and Resources** (August 6, 2024): A toolkit and resources for choosing the right LLM.

Note: The "Hacker News" mention is likely just for personal interest and not related to any specific project or announcement.